<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
Agglomerative Hierarchical Clustering
              
</p>
</div>

Data Science Cohort Live NYC July 2022
<p>Phase 4: Topic 35</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
    

In [ ]:
# Imports

import sys, os
import seaborn as sns
ex_path = os.path.abspath(os.pardir)
if ex_path not in sys.path:
    sys.path.append(ex_path)

from src.hier_example import *
from sklearn.datasets import make_blobs
from src.av_link_agglom_clust import centrAggClust as ac
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.spatial.distance import pdist
from sklearn.datasets import make_blobs, make_moons, load_iris
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import KernelDensity
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

from copy import deepcopy

%matplotlib inline

Often grouping within data is **hierarchical**:
- organisms: family/genus/species/subspecies/variants

- cosmic structure: superclusters, local superclusters, clusters/groups, galaxies

<center><img src = "Images/superclster.png" width = 800/></center>
<center> Laniakea galactic supercluster </center>

In many cases: 
- only have data
- want to discover multilevel cluster hierarchy

KMeans was not designed for this task:
- "Flat" clustering vs discovering group hierarchies.

#### Agglomerative Hierarchical Clustering

Idea is:
- Group nearest neighbor members
- Based on some similarity/distance scheme.
- Then repeat.
- Finds higher order grouping at each stage.

<center><img src = "Images/hierarch.gif" /></center>
<center> Agglomerates at each stage. </center>

Clearly a bottom-up approach:
- Use data, metric/agglomeration scheme to discover hierarchies.

#### Decision making on cluster merging

Distance notions:
- Euclidean or Manhattan distance
- Hamming distance
- Damerau-Levenshtein distance


String comparison: Damerau-Levenshtein distance
- Number of operations: insertions, deletions, substitutions 
- Transposition of two adjacent characters.

<img src = "Images/damerau.gif" />


#### Merge criterion: cluster distance evaluation

Given a distance measure: still there are different ways to evaluate **cluster** distance.

- Known as **linkage** (i.e. how do we link clusters?)


Merge clusters with minimum linkage value

<img src = "Images/linkage_type.png" />

### Types of hierarchical agglomerative clustering 

The way that distance is measured between clusters is called the model's **linkage**.

- Single Linkage 
    -  Minimum pair-wise distance: for any two clusters, take one observation from each and determine their distance. Do this over and over, until you have identified the overall minimum pair-wise distance. 
- Complete Linkage
    -  Complete linkage may be defined as the furthest (or maximum) distance between two clusters. That is, all possible pairwise distances between elements (one from cluster A and one from B) are evaluated and the largest value is used as the distance between clusters A & B. This is sometimes called complete linkage and is also called furthest neighbor.
- Average Linkage
    - The distance between clusters is defined as the average distance between the data points in the clusters. 
- Ward Linkage
    -  Ward method finds the pair of clusters that leads to minimum increase in total within-cluster variance after merging at each step.

Each linkage method has it's uses:

[This article](https://towardsdatascience.com/understanding-the-concept-of-hierarchical-clustering-technique-c6e8243758ec) describes the pros and cons of each approach.

#### Agglomerative Hierarchical Clustering in Python
- See it in action

- Scipy has nice functionality for this.

In [ ]:
# defines metric and calculates pair-wise distance between points 
from scipy.spatial.distance import pdist 

#for reshaping distance matrix 
from scipy.spatial.distance import squareform


Load in some data

In [ ]:
import pandas as pd
agg_data = pd.read_csv('Data/Aggregation.txt', header = None, usecols = [0,1], delimiter = '\t')
agg_data.columns = ['X', 'Y']
agg_data.head()

In [ ]:
agg_data.info()

In [ ]:
agg_data.plot(x = 'X', y = 'Y', kind = 'scatter');

Some hierarchical grouping here.

At each step:

- algorithm calculates matrix of linkage distances between clusters

In [ ]:
#computes the pairwise distance and returns a condensed array
#removes diagonal terms and flattens array
condensed_dist = pdist(agg_data, metric = 'euclidean' ) 
condensed_dist

Inspect in familiar matrix form
- Each column(row) indexes cluster
- Starting: each data points is their own cluster.

In [ ]:
sqp = squareform(condensed_dist)

sqpdf = pd.DataFrame(sqp, index = agg_data.index,
            columns = agg_data.index)

sqpdf.head()

At each iteration, merges clusters with minimum pairwise distance:
- Merged cluster is new *supercluster*
- For $n$ datapoints continues this $n-1$ times.
- Entire sequence of merges gives us entire hierarchy of clusters.

<center><img src = "Images/hierarch.gif" /></center>
<center> Agglomerates at each stage. </center>

scipy's linkage function:
- linkage(condensed pair-wise distance, linkage method, metric)
- returns an entire sequence of cluster merges in agglomerative process.

In [ ]:
from scipy.cluster.hierarchy import linkage

In [ ]:
np_linkage = linkage(condensed_dist, method='ward', metric='euclidean' )

In [ ]:
cols = ['clst1', 'clst2', 'dist', 'num_points']
linkage_df = pd.DataFrame(np_linkage,
                          columns = cols)
linkage_df.index.name = 'merge'
linkage_df

Rows: Cluster merges

- 788 data points: 788 - 1 merges

- Columns: clusters merged, distance between clusters, number of points in new cluster

In [ ]:
print(linkage_df.shape)

#### Creating the dendrogram
- scipy's dendrogram function

In [ ]:
from scipy.cluster.hierarchy import dendrogram

In [ ]:
fig, ax = plt.subplots(figsize = (5,7))
dendrogram(linkage_df, no_labels = True, leaf_font_size = 8, color_threshold = 100,  orientation = 'left' )
ax.set_xlabel('Linkage distance')
plt.show()

At given linkage distance threshold:
- want to output "flattened" cluster assignments:
- scipy's fcluster function: fcluster(linkage_matrix, threshold, criterion)

In [ ]:
from scipy.cluster.hierarchy import fcluster
from copy import deepcopy

In [ ]:

# criterion as linkage distance is fairly common
clust_assgn = fcluster(linkage_df, t = 100, criterion = 'distance')

#put labels into dataframe and scatterplot
agg_assigned = deepcopy(agg_data)
agg_assigned['label'] = clust_assgn

In [ ]:
agg_assigned.head()

Scatterplot with hue on cluster assignments

In [ ]:
n_labels = len(agg_assigned['label'].unique())
fig, ax = plt.subplots(figsize = (6,4))
sns.scatterplot(x = 'X', y = 'Y',
                hue = 'label', 
                data = agg_assigned, 
                palette=sns.color_palette(
                    'tab10', n_labels), ax = ax)

plt.show()

In [ ]:
def plot_with_threshold(data, linkage, threshold = 100):
    
    clust_assgn = fcluster(linkage, t = threshold, criterion = 'distance')
    
    agg_assigned = deepcopy(data)
    agg_assigned['label'] = clust_assgn
    n_labels = len(agg_assigned['label'].unique())
    

    fig, ax = plt.subplots(1,2, figsize = (8,5))
    d = dendrogram(linkage, no_labels = True, leaf_font_size = 8,
               color_threshold = threshold, orientation = 'left', ax = ax[0] )
    ax[0].set_xlabel('Linkage distance')
    
    ax[0].axvline(threshold, c = 'b', linestyle = '--')
    
    #NEW CODE
    ax[1].scatter(agg_assigned['X'].iloc[d['leaves']], 
                  agg_assigned['Y'].iloc[d['leaves']], 
                  color=d['leaves_color_list'])

    plt.show()

In [ ]:

plot_with_threshold(agg_data, linkage_df, 50)

#### Evaluating the agglomerative linkage and dendrogram

**Cophenetic correlation coefficient**

- correlation between point pairwise distance and linkage distance at which pairs are joined.

- how closely related is the agglomerative clustering to the underlying data structure?


- Is a correlation coefficient: 
    - 1 = perfect correlation
    - 0 = no correlation
    - -1 = anti-correlation

#### Cophenetic distances

- Taking point $i$ and $j$: 
- linkage distance between respective clusters before merge.



<center><img src = "Images/cophen_dist.jpg" width = 400/></center>

In [ ]:
from scipy.cluster.hierarchy import cophenet

Takes in linkage matrix and reduced pairwise distance matrix:
- returns correlation and list of cophenetic distances.

In [ ]:
# evaluate effectiveness of hierarchical clustering with cophenetic correlation
corr_coef, cophen_dist = cophenet(np_linkage, condensed_dist)
print(corr_coef)

The cophenetic correlation is measuring how effective our agglomerative clustering is at capturing the entire **hierarchy**.

#### Hierarchical clustering with `sklearn` 

- Scipy has much better functionality for exploring full hierarchy
- Sklearn for using hierarchical clustering in flat clustering/prediction tasks
- can specify number of clusters, adjusts threshold automatically.


In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
# affinity is distance metric(euclidean, manhattan)
# linkage (average, complete, single, ward, etc.)
agg_cluster = AgglomerativeClustering(n_clusters = 7,
                                      affinity = 'euclidean', 
                                      linkage = 'average')
agg_cluster

In [ ]:
sns.scatterplot(x = 'X', y = 'Y', data = agg_data)
plt.show()

In [ ]:
agg_labels = agg_cluster.fit_predict(agg_data)
sklearn_agg_data = deepcopy(agg_data)
sklearn_agg_data['labels'] = agg_labels
sklearn_agg_data.head()

In [ ]:
sns.scatterplot(x = 'X', y = 'Y', 
                hue = 'labels', palette = 'tab10',
                data = sklearn_agg_data)
plt.show()

Benefit of sklearn implementation:
- Chooses distance threshold for you given number of clusters.

#### Evaluation

Operating in this flat mode:
- getting reasonable cluster number -- silhouette score.
- not a sure fire method but not terrible either.

In [ ]:
def plot_silh_scores(X, K, increment, linkage):

    klist = np.arange(2,K,increment)
    score_list = []
    for k in klist:
        agg_clstrer = AgglomerativeClustering(n_clusters = k,
                                      affinity = 'euclidean', 
                                      linkage = linkage)
        
        clstr_labels = agg_clstrer.fit_predict(X)

        score = silhouette_score(X, clstr_labels)
        score_list.append(score)
        
    sns.lineplot(x = klist, y = score_list, color = 'r')
    plt.ylabel('Silhouette Coefficient')
    plt.xlabel('K')
    plt.title('Silhouette coefficient plot')
    plt.show()

In [ ]:
plot_silh_scores(agg_data, 15, 1, 'average')

Under average linkage:
- k = 6 might be a good choice.

In [ ]:
def plot_with_sklearn(X, k, linkage):
    agg_cluster = AgglomerativeClustering(n_clusters = k,
                                      affinity = 'euclidean', 
                                      linkage = linkage)
    
    agg_labels = agg_cluster.fit_predict(X)
    sklearn_agg_data = deepcopy(X)
    sklearn_agg_data['labels'] = agg_labels
    
    sns.scatterplot(x = 'X', y = 'Y', 
                hue = 'labels', palette = 'tab10',
                data = sklearn_agg_data)
    plt.show()


- Seven is actually the optimal number for the flat clustering.
- Clustering is a tricky business and often requires human judgement.
- Sklearn's API makes it easy for agglomerative clustering: don't have to mess with thresholds.

In [ ]:
plot_with_sklearn(agg_data, 6, 'average')

In [ ]:
plot_with_sklearn(agg_data, 7, 'average')

But:

- agglomerative hierarchical clustering for flat clustering tasks: SLOW.
- designed to calculate whole hierarchy.
- better methods for flat clustering: kmeans, DBSCAN, mixture methods, variational autoencoders, etc.

## Advantages & Disadvantages of hierarchical clustering

#### Advantages
- Intuitive and easy to implement
- More informative than k-means because it takes individual relationship into consideration
- Allows us to look at dendrogram and decide number of clusters

#### Disadvantages
- Very sensitive to outliers
- Cannot undo the previous merge, which might lead to problems later on 

### Further reading

- [from MIT on just hierarchical](http://web.mit.edu/6.S097/www/resources/Hierarchical.pdf)
- [from MIT comparing clustering methods](http://www.mit.edu/~9.54/fall14/slides/Class13.pdf)
- [fun CMU slides on clustering](http://www.cs.cmu.edu/afs/andrew/course/15/381-f08/www/lectures/clustering.pdf)